
# Storytime


Before running or if using a different scenario, the user needs to  
make the below changes.  
* Open the scenario 
* Update the file path
* Set the animation time & time step
* In createfolders() decide how the files should be structured  

The goal in this script is to simplify generating eoir synthetic data  
This is done in a few steps 

## _main()_

main() kicks everything off. Using the scenario time, it calculates the  
number of total frames between start and stop. This will be the number  
of times eoirdata() steps through the scenario. The loop jumps to the  
beginning of the scenario then cycles through the following cloud and  
atmospheric parameters.   

|Clouds| . |Atmosphere|
|:-----|:-:|---------:|
|Off   | . |Off       |
|On    | . |Off       |
|On    | . |Simple       |
|On    | . |MODTRAN   |

Once the parameters are set, it dives into eoirdata()

## _eoirdata()_  

eoirdata() uses connect commands to generate the bitmap and the  
raw data. In the same line, the data is stored into it's respective  
folder (see createfolders()). This function then steps forward in time  
and does the whole thing again. The range, or number of iterations is   
set by the number of frames. Once it reaches the end time the function  
jumps out. 

## _createfolders()_  
Create the folders that eoirdata() will save to. This only needs to be run once.  
Kept separate so I can quickly clear/delete folders and regenerate them

## _Misc_

Replace the manually entered information and let that be entered in as as function

if using custom file, then use  
root.ExecuteCommand('EOIR */ CloudData SetValue MyConstDataSet Altitude 12    
PercentCloudCoverConst 1 PercentCloudCoverFile   
"C:\ProgramData\AGI\STK 12\STKData\VO\Clouds\clouds_2020126.100000_2020127.160000.cld  
EmissivityConst .5 TemperatureConst 233 ForwardScatterConst .1)  


* attach to scenario PID  


# Connect Command Reference

Atmosphere Key
* 0 - off.
* 1 - Simple atmosphere.
* 2 - MODTRAN atmosphere.

Atmosphere Simple Parameters
* Aerosol Models: Rural
* Visibility: 25.00 km
* Humidity: 45.8 %
    
Cloud Key
* On - turns on clouds.
* Off - turns off clouds.

Cloud Parameters
* Altitude: 12 km
* Forward Scatter: 10%
* Back Scatter: 80%
* Constant Coverage: 75%
* Constant Emissivity and Temperature
* Emissivity: 50%
* Temperature: 233K


In [4]:
import os 
import csv
import time
import itertools
from datetime import timedelta
from datetime import datetime
from agi.stk12.stkdesktop import STKDesktop
from agi.stk12.stkobjects import *

# Open EOIR_Clouds_AirToAir.sc
# Connect to the scenario
# Get the IAgStkObjectRoot interface 

stk      = STKDesktop.AttachToApplication()  
root     = stk.Root                           
scenario = root.CurrentScenario 

# Set the scenario animation properties
animation               = scenario.Animation
animation.StartTime     = '5 May 2020 11:41:30.000'
animation.EnableAnimCycleTime = True
animation.AnimCycleTime = '5 May 2020 11:46:30.000'
animation.AnimCycleType = AgEScEndLoopType.eLoopAtTime
animation.AnimStepValue = 0.5


In [ ]:
def createfolders():

    data    = ['raw','processed'] #  the state of the data
    sensor  = ['LWIR','Visible']  #  names of the sensors in the scenario
    ftype   = ['bitmap','txt']    #  exported data format

    cld     = ['Off','On','On','On']
    atm     = [0,0,1,2]

    # File path for the output data 
    path = "C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/"
    # newdir = os.mkdir("{}output/".format(path))

    # Directory structure based on generated data
    for (a,b,c) in itertools.product(data,ftype,sensor): 
        for (x,y) in itertools.zip_longest(cld,atm):
            name = "{}_{}_{}_Cld{}_Atm{}".format(a, b, c, x, y)
            #print("{}output/{}".format(path,name))
            os.mkdir("{}output/{}".format(path,name))

createfolders()

In [ ]:
def eoirdata(x,y,z):

    # EOIR connect commands SaveSceneImage & SaveSceneRawData 
    command_bmp_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneImage "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output/raw_bitmap_LWIR")
    command_txt_LWIR    = ("EOIRDetails */Aircraft/SouthBound/Sensor/LWIR SaveSceneRawData "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output/raw_txt_LWIR")
    command_bmp_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneImage "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output/raw_bitmap_Visible")
    command_txt_Visible = ("EOIRDetails */Aircraft/SouthBound/Sensor/Visible SaveSceneRawData "\
        "\"C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output/raw_txt_Visible")
    
    # Loop through the entire scenario 
    for i in range(z):
        # print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        # print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        # print("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        # print("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_LWIR,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_LWIR,x,y,i))
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.bmp\"".format(command_bmp_Visible,x,y,i))    
        root.ExecuteCommand("{}_Cld{}_Atm{}/{}.txt\"".format(command_txt_Visible,x,y,i))
        root.StepForward()



In [5]:
def main():

    # Measure elapsed time for data generation
    start = time.time() 
    
    # Total frames generated based on scenario animation time/timestep
    d1 = datetime.strptime(animation.StartTime,     "%d %b %Y %H:%M:%S.%f")
    d2 = datetime.strptime(animation.AnimCycleTime, "%d %b %Y %H:%M:%S.%f")
    k  = int(timedelta.total_seconds(d2 - d1)/animation.AnimStepValue)
    #k  = 60 # run for testing

    cld     = ['Off','On','On','On']
    atm     = [0,0,1,2]
    
    # Jump to the beginning of the animation time
    # Set EOIR Cloud and Atmosphere properties
    # Animate and generate data for each timestep 
    for (i,j) in itertools.zip_longest(cld,atm):
        root.Rewind()
        root.ExecuteCommand("EOIR */ SetAtmosphere Clouds {}".format(i))
        root.ExecuteCommand("EOIR */ SetAtmosphere Mode {}".format(j))
        eoirdata(i,j,k)

    end = time.time()
    print("Elapsed Time (hr): " + str((end - start)/(3600))
if __name__ == '__main__':
    main()  

Elapsed Time: 515.6723907073339


In [5]:
import os.path

path = 'C:/Users/nkazmi/Documents/STK 12/EOIR_Clouds_AirToAir - Python - new flight/output'

isExist = os.path.exists(path) 
print(isExist)
if isExist == False:
    createfolders()
else:
    print('Directories Exist')
    

True
continue
